Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [2]:
importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
ops0=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3Planes.npy')

SaveFolder=FBS.suite2pInitiate(ops0)
binFile = glob.glob(ops0['save_path0'] + '/*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
##Load Data
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)

Processed data would be saved inF:\LuSLMOnlineTest\04222024\suite2p
Total Frames 21600 is found


Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [3]:
rawBin.shape

(21601, 512, 512)

In [3]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Binning movie in chunks of length 06
Binned movie of size [1200,512,512] created in 7.33 sec.
Binned movie denoised (for cell detection only) in 22.04 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 10.00 
0 ROIs, score=108.41
Detected 65 ROIs, 19.63 sec
After removing overlaps, 64 ROIs remain
added enhanced mean image
Masks created, 0.57 sec.


c:\users\user\suite2p\suite2p\extraction\extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 64 ROIs in 7200 frames, 13.44 sec.
Extracted fluorescence from 64 ROIs in 7200 frames, 8.67 sec.
['compact', 'skew', 'npix_norm']
Processing plane 1
Binning movie in chunks of length 06
Binned movie of size [1200,512,512] created in 7.43 sec.
Binned movie denoised (for cell detection only) in 22.75 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 10.00 
0 ROIs, score=83.70
Detected 77 ROIs, 19.33 sec
After removing overlaps, 75 ROIs remain
added enhanced mean image
Masks created, 0.22 sec.
Extracted fluorescence from 75 ROIs in 7200 frames, 8.40 sec.
Extracted fluorescence from 75 ROIs in 7200 frames, 8.45 sec.
['compact', 'skew', 'npix_norm']
Processing plane 2
Binning movie in chunks of length 06
Binned movie of size [1200,512,512] created in 7.82 sec.
Binned movie denoised (for cell detection only) in 24.21 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 10.00 
0 ROIs, score=76.38
Detected 64 ROIs, 19.37 sec
Af

[None, None, None]

In [4]:
FBS.CombinePlanes(SaveFolder,ops0)

appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.


Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [5]:
FBS.PostMannual(SaveFolder,ops0)

201 of units including 36 identified cells in combined planes
